In [ ]:
import numpy as np
import pandas as pd
import re
import csv
import os
import json
from functools import reduce
from dateutil.relativedelta import relativedelta
from datetime import datetime
from pathlib import Path
Path("../results").mkdir(parents=True, exist_ok=True)

In [ ]:
def load_data(dataset):
        print('----Started data loading process...')
        df = pd.read_csv(dataset)
        print('----Data loaded.')
        return df

## Create HPI Summarization Segmentation CSV

In [ ]:
notes_csv = '../data/mimic-III-dataset/NOTEEVENTS.csv'
hc_csv = '../data/hpi-dataset/HOSPITAL_COURSES_FIRSTLAST.csv'
admit_csv = '../data/mimic-III-dataset/ADMISSIONS.csv'
pt_csv = '../data/mimic-III-dataset/PATIENTS.csv'

In [ ]:
df_notes = load_data(notes_csv)
df_hc = load_data(hc_csv)
df_chart = load_data(admit_csv)
df_pt = load_data(pt_csv)

In [ ]:
gen_admit_notes = ['Admission Note', 'Attending Admission Note', 'CCU - Admit Note, History and Physical', 'CCU Admission Note', 'CCU Fellow Admit Note', 'CCU Resident Admission Note', 'EP Admit H & P', 'ICU Admission Note', 'ICU Atending Progress note', 'ICU Attdg Admit', 'ICU Attdg Admit Note', 'ICU Attending Admission Note', 'ICU Attending Admission Physician Note', 'ICU Attending Admit Note', 'MICU Admission', 'MICU Attdg Admit Note', 'MICU Attending Admission Note', 'MICU Attending Admit Note', 'MICU Resident Admission Note', 'RESIDENT ADMISSION NOTE', 'Resident Admission H&P', 'Resident Admission Note', 'Resident Admission/Observation Note', 'Resident H&P', 'Resident/Attending Admission H&P', 'Trauma SICU admission note', 'critical care admit note', 'micu attending admission note', 'micu staff admission note']
phys_admit_notes = ['Admission H&P', 'Admission Note', 'Admission Note - MICU', 'Admission Note - Surgical Critical Care', 'Admit Note', 'Attending / Resident Admission Note', 'Attending Admission Note', 'Attending Admit Note', 'CCU Admission Note', 'CCU Admit note', 'CCU Admit: Cardiology Comprehensive Physician Note', 'CCU FELLOW ADMISSION NOTE', 'CCU Fellow Admission Note', 'Cardiology Fellow Admission', 'Cardiology Fellow Admission Note', 'Cardiology Fellow Admission Note Admission', 'Cardiology Fellow Admit', 'Cardiology Fellow Admit Addendum', 'Cardiology Fellow Admit Note', 'Cardiology Fellow CCU Admission Note', 'Cardiology Fellow CCU Admission Noted', 'Cardiology Fellow CCU admission note', 'Cardiology fellow Admission Note', 'Cardiology fellow CCU Admit note', 'Cardiology fellow CCU admission note', 'Cardiology fellow CCU admit note', 'Cardiology fellow H&P', 'Critical Care Attending Admit Note', 'Critical Care admission note', 'Critical Care attending admission', 'Critical care admit note', 'Critical care attending admission note', 'Critical care attending admit', 'Critical care staff admission note', 'Critical care staff admit note', 'EP Admission Note', 'Fellow / Physician Attending Admission Note - MICU', 'Fellow Admission', 'Fellow Admission note', 'Fellow Admit', 'Fellow admission Note', 'ICU Admission Note', 'ICU Attending Admission Note', 'ICU Attending Admit Note', 'ICU Attending Note Admission Note', 'ICU Fellow Admission Note - MICU', 'ICU Fellow/Attending Admission Note - MICU', 'ICU Resident Admission Note', 'ICU resident admission note', 'MICU ADMISSION NOTE', 'MICU Admission', 'MICU Admission Note', 'MICU Fellow Admission Note', 'MICU Overnight Intensivist Fellow Admission', 'MICU Resident / Attending Admission Note', 'MICU Resident Admission Note', 'MICU Resident/Attending Admission Note', 'MICU attending admission', 'MICU attending admission note', 'MICU attending admit note', 'MICU staff admit note', 'MS4 Admission Note', 'Medical Student/Attending Admission Note - MICU', 'Overnight Intensivist (Fellow) Admit', 'Overnight Intensivist / Fellow Admission', 'Overnight Intensivist Admission', 'Overnight Intensivist Admit', 'Overnight Intensivist Fellow Admission', 'Overnight Intensivist Fellow Admission Note - MICU', 'Overnight intensivist admission', 'Physican Resident Admission Note', 'Physican Resident Admission Note - MICU attending', 'Physician Admission Note', 'Physician Admission Note - MICU', 'Physician Attending  Admission Note', 'Physician Attending - Resident Admission Note - MI', 'Physician Attending / Resident Admission Note - MI', 'Physician Attending Admission Note', 'Physician Attending Admission Note - CCU', 'Physician Attending Admission Note - Howell', 'Physician Attending Admission Note - MICU', 'Physician Attending Admission Notel', 'Physician Attending Admit Note - MICU', 'Physician Attending Resident Admission Note - MICU', 'Physician Attending/ Resident Admission Note - MIC', 'Physician Attending/Resident Admission Note - MICU', 'Physician Fellow/Attending Admission Note - MICU', 'Physician ICU  Admission Note', 'Physician ICU Admission Note', 'Physician Resident (Attending Addendum) Admission', 'Physician Resident / Attending Admission Note', 'Physician Resident / Attending Admission Note - MI', 'Physician Resident / attending Admission Note - MI', 'Physician Resident /Attending Admission Note - MIC', 'Physician Resident /attending Admission Note - MIC', 'Physician Resident Admission Note', 'Physician Resident Admission/Progress Note - CCU', 'Physician Resident Admission/Transfer Note - MICU', 'Physician Resident Attending/Admission Note - MICU', 'Physician Resident and Attending  Admission Note -', 'Physician Resident and Attending Admission Note', 'Physician Resident and Attending Admission Note -', 'Physician Resident and Attg Admission Note - MICU', 'Physician Resident and attedning Admission Note -', 'Physician Resident and attg Admission Note - MICU', 'Physician Resident and fellow Admission Note', 'Physician Resident/ attending Admission Note - MIC', 'Physician Resident/Attending Admission Note', 'Physician Resident/Attending Admission Note - MICU', 'Physician Resident/Attening Admission Note - MICU', 'Physician Resident/ICU Attending Admission Note -', 'Physician Resident/attending  Admission Note - MIC', 'Physician Resident/attending Admission Note - MICU', 'Physician Residentand Attending  Admission Note -', 'Physician Resodent Admission Note - MICU', 'Physician Surgical Admission Note', 'Physician/Attending Resident Admission Note', 'RESIDEN ADMISSION NOTE', 'RESIDENT ADMISSION NOTE', 'Resident / Attending Admission Note', 'Resident / Attending Admission Note - MICU', 'Resident / Attending Admission Notes', 'Resident / Attending Admission notes', 'Resident / Attending Admit Notes', 'Resident / Attending admission note', 'Resident Admission Note', 'Resident Admission Note - CCU', 'Resident Admission Note / Attending Addendum', 'Resident Admission Note-critical care addendum', 'Resident Admission Note/Attending Addendum', 'Resident Admission note', 'Resident Admission note - TSICU', 'Resident Admission/ Attending Admission note', 'Resident Admit Note / Attending Addendum', 'Resident admission note', 'Resident and Attending Admission Notes - MICU', 'Residnet / Attending Admission Notes', 'SICU admission note', 'SICU resident admission note', 'Sub-Intern (4th year med student) Admission Note', 'TSICU Resident Admission Note', 'admission note', 'critical care admit note', 'critical care attending admission note', 'critical care staff admission note', 'resident admission note']

In [ ]:
print(len(gen_admit_notes) + len(phys_admit_notes))

In [ ]:
df_admit_notes = df_notes[df_notes['DESCRIPTION'].isin(gen_admit_notes + phys_admit_notes)]
df_admit_notes = df_admit_notes.drop_duplicates('HADM_ID') #only keep one admit note for each patient
df_admit_notes.reset_index(drop=True, inplace=True)
df_admit_notes = df_admit_notes.rename(columns={"DESCRIPTION": "ADMIT_NOTE_DESCRIPTION", "TEXT": "ADMIT_NOTE_TEXT"})

In [ ]:
# Only save the HPI portion of the Admit Note
df_admit_notes['ADMIT_NOTE_TEXT'] = df_admit_notes['ADMIT_NOTE_TEXT'].apply(lambda x: " ".join(x.split()))
df_admit_notes['HPI_TEXT'] = df_admit_notes['ADMIT_NOTE_TEXT'].apply(lambda x: ' '.join(re.findall('(?<=HPI:).*?(?=[A-Z]+\:)', x)))
df_admit_notes = df_admit_notes[(df_admit_notes["HPI_TEXT"] != ' ') & (df_admit_notes["HPI_TEXT"] != '') & (df_admit_notes["HPI_TEXT"].str.len() > 100)]
df_admit_notes.reset_index(drop=True, inplace=True)

In [ ]:
df_chart_full = df_chart.merge(df_pt, on="SUBJECT_ID", how="left", suffixes=('', '_y'))

In [ ]:
hc_cols = ['ROW_ID', 'SUBJECT_ID', 'HADM_ID', 'CHARTDATE', 'HOSPITAL_COURSE', 'TOKENIZED_HOSPITAL_COURSE', 'HC_HPI']
admit_cols = ['HADM_ID', 'ADMIT_NOTE_DESCRIPTION', 'ADMIT_NOTE_TEXT', 'HPI_TEXT']
chart_cols = ['HADM_ID', 'ADMISSION_TYPE', 'ADMISSION_LOCATION', 'MARITAL_STATUS', 'ETHNICITY', 'DIAGNOSIS', 'GENDER', 'DOB', 'HOSPITAL_EXPIRE_FLAG']

In [ ]:
dfs = [df_hc[hc_cols], df_admit_notes[admit_cols], df_chart_full[chart_cols]]

In [ ]:
df_final = reduce(lambda left,right: pd.merge(left,right,on='HADM_ID'), dfs)

In [ ]:
df_final["DOB"] = df_final["DOB"].str.split(' ').str[0]

In [ ]:
df_final["AGE"] = df_final.apply(lambda row: relativedelta(datetime.strptime(row['CHARTDATE'], '%Y-%m-%d'), datetime.strptime(row['DOB'], '%Y-%m-%d')).years, axis=1)
for index, row in df_final.iterrows():
  if row['AGE'] > 89:
    df_final.loc[index,'AGE'] = np.nan
df_final = df_final.drop(columns=['DOB'])
df_final = df_final.sort_values(by=['ROW_ID'])
df_final.reset_index(drop=True, inplace=True)

In [ ]:
df_final.to_csv('../data/hpi-dataset/SUMMARIZATION_HPI.csv', index=False)

## NLP with HPI Section

In [ ]:
hpi_csv = '../data/hpi-dataset/SUMMARIZATION_HPI.csv'

In [ ]:
df_hpi = load_data(hpi_csv)

In [ ]:
# Create CSV for Transformer
# ROW ID, SUBJECT_ID, HADM_ID, TEXT, SUMMARY

# Get Sentence with dates in them
rows_list = []
for i,row in df_hpi.iterrows():
  age = str(int(row['AGE'])) if not np.isnan(row['AGE']) else 'NONE'
  expired = 'YES' if row['HOSPITAL_EXPIRE_FLAG'] else 'NO'
  # compile NLP note text
  text = "CATEGORY: " + row['ADMIT_NOTE_DESCRIPTION'] + ", " 
  text += "TYPE: " + row['ADMISSION_TYPE'] + ", " 
  text += "ADMIT LOCATION: " + row['ADMISSION_LOCATION'] + ", "
  text += "AGE: " + age + ", "
  text += "GENDER: " + row['GENDER'] + ", "
  text += "MARITAL STATUS: " + str(row['MARITAL_STATUS']) + ", "
  text += "ETHNICITY: " + row['ETHNICITY'] + ", "
  text += "ADMIT DX: " + row['DIAGNOSIS'] + ", "
  text += "ADMIT TEXT: " + row['HPI_TEXT'].strip()
  dict1 = {'DS_ID': row['ROW_ID'],
          'SUBJECT_ID': row['SUBJECT_ID'],
           'HADM_ID': row['HADM_ID'],
           'DATE': row['CHARTDATE'],
           'DIAGNOSIS': row['DIAGNOSIS'],
           'EXPIRED': expired,
           'TEXT': text,
           'SUMMARY': row['HC_HPI'],
           }
  rows_list.append(dict1)

df_hpi_nlp = pd.DataFrame(rows_list) 
df_hpi_nlp.insert(loc=0, column='ID', value=df_hpi_nlp.index + 1)

In [ ]:
# Create Train, Validate, and Test set with 90/10/10 split
train, validate, test = np.split(df_hpi_nlp.sample(frac=1, random_state=42),[int(.8*len(df_hpi_nlp)), int(.9*len(df_hpi_nlp))])
train = train.sort_values(by='ID').reset_index(drop=True)
validate = validate.sort_values(by='ID').reset_index(drop=True)
test = test.sort_values(by='ID').reset_index(drop=True)

In [ ]:
from pathlib import Path
Path("../data/hpi-dataset/summarization-datasets").mkdir(parents=True, exist_ok=True)

In [ ]:
df_hpi_nlp.to_csv('../data/hpi-dataset/summarization-datasets/HPI_FULL.csv', index=False)
train.to_csv('../data/hpi-dataset/summarization-datasetsHPI_TRAIN.csv', index=False)
validate.to_csv('../data/hpi-dataset/summarization-datasets/HPI_VALIDATE.csv', index=False)
test.to_csv('../data/hpi-dataset/summarization-datasets/HPI_TEST.csv', index=False)

In [ ]:
df_hpi_nlp.head()

# Dataset Statistics

In [ ]:
hpi_full = '../data/hpi-dataset/summarization-datasets/HPI_FULL.csv'
df_hpi_full = load_data(hpi_full)

In [ ]:
total_docs = len(df_hpi_full)
text_count = df_hpi_full['TEXT'].str.split().str.len()
summary_count = df_hpi_full['SUMMARY'].str.split().str.len()

In [ ]:
print("Total Docs: ", total_docs)
print("Text Mean: ", text_count.mean())
print("Summary Mean: ", summary_count.mean())

## Transformers

In [ ]:
import torch
import datasets
from datasets import load_dataset, load_metric
from transformers import (BartModel, BartTokenizer, BartForConditionalGeneration, 
BartConfig, Seq2SeqTrainer, Seq2SeqTrainingArguments)
from transformers import Trainer, TrainingArguments
from transformers.models.bart.modeling_bart import shift_tokens_right

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(device)

In [ ]:
train_file = '../data/hpi-dataset/summarization-datasets/HPI_TRAIN.csv'
validation_file = '../data/hpi-dataset/summarization-datasets/HPI_VALIDATE.csv'
test_file = '../data/hpi-dataset/summarization-datasets/HPI_TEST.csv'

### BART Model

In [ ]:
Path("../models/HPI-BART").mkdir(parents=True, exist_ok=True)

In [ ]:
# RUN BART
# Original Model: facebook/bart-large-cnn
# Original Tokenizer: facebook/bart-large
%run "../transformers/seq2seq/run_summarization.py" \
--model_name "facebook/bart-large-cnn" \
--tokenizer_name "facebook/bart-large" \
--output_dir "../models/HPI-BART" \
--overwrite_output_dir True \
--train_file "../data/hpi-dataset/summarization-datasets/HPI_TRAIN.csv" \
--validation_file "../data/hpi-dataset/summarization-datasets/HPI_VALIDATE.csv" \
--test_file "../data/hpi-dataset/summarization-datasets/HPI_TEST.csv" \
--text_column TEXT \
--summary_column SUMMARY \
--max_source_length 1024 \
--max_target_length 150 \
--num_train_epochs 3 \
--num_beams 4 \
--per_device_train_batch_size=3 \
--per_device_eval_batch_size=3 \
--do_train \
--do_eval \
--do_predict \
--predict_with_generate True